In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def rot_aut(Xtr,ytr):
    # aug on train
    data_cnt = len(ytr)
    print(data_cnt)
    aug_X = []
    aug_y = []

    for i in range(data_cnt):
        img = Xtr[i]
        tmp_y = ytr[i]

        # org img
        aug_X.append(img)
        aug_y.append(tmp_y)

        # flip
        tmp_img = np.fliplr(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.flipud(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.rot90(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)
    return np.array(aug_X),np.array(aug_y)

In [ ]:
from keras.callbacks import ModelCheckpoint
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')

model model


In [ ]:
from sklearn.model_selection import KFold
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        curr_x,curr_y = rot_aut(curr_x,curr_y)
        val_x,val_y = train_x[test_index],y[test_index]
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        model.fit(curr_x,curr_y,
                  validation_data=(val_x,val_y),
                  batch_size=64, epochs=3, 
                  verbose=2,
                  callbacks=[model_chk]
                 )
        model = load_model(model_p)
        
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4,rnd=1)

1203
Train on 4812 samples, validate on 401 samples
Epoch 1/3
 - 5s - loss: 0.6608 - acc: 0.6095 - val_loss: 0.6278 - val_acc: 0.6608

Epoch 00001: val_loss improved from inf to 0.62783, saving model to best_m.h5
Epoch 2/3
 - 2s - loss: 0.5590 - acc: 0.7126 - val_loss: 0.5431 - val_acc: 0.6783

Epoch 00002: val_loss improved from 0.62783 to 0.54309, saving model to best_m.h5
Epoch 3/3
 - 3s - loss: 0.4904 - acc: 0.7589 - val_loss: 0.4749 - val_acc: 0.7531

Epoch 00003: val_loss improved from 0.54309 to 0.47485, saving model to best_m.h5
1203
Train on 4812 samples, validate on 401 samples
Epoch 1/3
 - 3s - loss: 0.6727 - acc: 0.5856 - val_loss: 0.5940 - val_acc: 0.7207

Epoch 00001: val_loss improved from inf to 0.59396, saving model to best_m.h5
Epoch 2/3
 - 3s - loss: 0.5989 - acc: 0.6706 - val_loss: 0.5529 - val_acc: 0.7581

Epoch 00002: val_loss improved from 0.59396 to 0.55291, saving model to best_m.h5
Epoch 3/3
 - 3s - loss: 0.5523 - acc: 0.7057 - val_loss: 0.4836 - val_acc: 0.79